In [1]:
from pyspark import SparkContext

In [2]:
sc=SparkContext() #there can't be multiple sparkcontext in one file

jupyter notebook command to create and write a textfile

In [11]:
%%writefile example.txt
first line 
second line
Third line
fourth line

Overwriting example.txt


In [12]:
textfile=sc.textFile('example.txt') #textFile object created is an RDD object

**Actions**

In [14]:
textFile.count() #returns number of lines(elements)here

4

In [15]:
textfile.first()

'first line '

In [16]:
textfile.collect()

['first line ', 'second line', 'Third line', 'fourth line']

In [20]:
textfile.take(3)

['first line ', 'second line', 'Third line']

**Transformations**

In [21]:
secfind=textfile.filter(lambda line:'second' in line)

In [26]:
secfind #it's just a recipe that's being created so it won't take much time.It's only executed when we run some action command

PythonRDD[13] at RDD at PythonRDD.scala:48

In [23]:
secfind.collect()

['second line']

In [24]:
secfind.count()

1

In [28]:
%%writefile example2.txt
first
second line
the third line
then the fourth line

Writing example2.txt


In [32]:
text_rdd=sc.textFile('example2.txt')

In [34]:
words=text_rdd.map(lambda line:line.split())

In [35]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'the', 'fourth', 'line']]

In [36]:
text_rdd.flatMap(lambda line:line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'the',
 'fourth',
 'line']

In [37]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [38]:
services=sc.textFile('services.txt')

In [41]:
services.take(3)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00']

In [40]:
services.map(lambda line:line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [43]:
#removing hashtag
services.map(lambda line:line[1:] if line[0]=='#' else line).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [46]:
clean=services.map(lambda line:line[1:] if line[0]=='#' else line)
clean=clean.map(lambda line:line.split())

In [47]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [48]:
 clean.map(lambda lst:(lst[3],lst[-1])).collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [49]:
#step1
pairs=clean.map(lambda lst:(lst[3],lst[-1]))

In [52]:
rekey=pairs.reduceByKey(lambda amt1,amt2:amt1+amt2)

In [53]:
rekey.collect()

[('State', 'Amount'),
 ('NY', '100.00750.00'),
 ('TX', '450.00200.00'),
 ('CA', '200.00500.00')]

In [56]:
#step2
rekey=pairs.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [61]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [64]:
rekey.filter(lambda x:not x[0]=='State').collect()

[('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [65]:
#step3
rekey_lst=rekey.filter(lambda x:not x[0]=='State')

In [68]:
rekey_lst.sortBy(lambda stAmt:stAmt[1],ascending=False).collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [69]:
#step4
sorted_rekey_lst=rekey_lst.sortBy(lambda stAmt:stAmt[1],ascending=False)

In [70]:
#step5
sorted_rekey_lst.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [71]:
x=['ID','State','Amount']

In [72]:
def func1(tup):
    #tuple unpacking
    (Id,st,amt)=tup
    return amt

In [74]:
func1(x)

'Amount'

In [75]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [79]:
clean.sample(True,0.50).collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00']]

In [96]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [97]:
samp1=clean.sample(True,0.50) #transformation
samp1.collect() #since withreplacement is set true,we can see the entries are repeated

[['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00']]

In [98]:
clean.takeSample(withReplacement=True,num=3) #action

[['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00']]

In [99]:
samp2=clean.sample(withReplacement=True,fraction=0.25)
samp2.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00']]

In [100]:
uni=samp1.union(samp2)

In [101]:
uni.collect()

[['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00']]

In [108]:
samp3=clean.sample(withReplacement=False,fraction=0.75)

In [109]:
samp3.collect()

[['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]